# Financial Markets - Practical Lesson 4

## Recap

* Basic Python (mostly not related directly to finance)
* How to implement a discount factor interpolation function
* Wrapping up functionality in **classes** in order to work with multiple data sets more easily
* Libor forward rate calculator

## Today's lesson

We're going to look at:

* Modules, and start building up our library of finance-related functionality
* Implementing an Overnight Index Swap class for calculating the NPV of an OIS.

## Modules

Notebooks are great for interactive exploratory use, but once you have some code (i.e. functions or classes) which you'd like to reuse often, rather than copy/pasting it into every notebook, you can save it in a `.py` file and use it from your notebook.

These work just like the modules we have been importing up to now, except they're written by us! Take a look at [this video](https://www.youtube.com/watch?v=AqCl65wxikw) for an example of how it's done.

We're going to start writing a module called `finmarkets`, and over the course of the remaining lessons we'll add functionality related to the theory lessons.

Make sure that the `finmarkets.py` file is in the same folder as this notebook, otherwise the import will fail. Remember that if you make any changes to the module, you'll have to restart the Python kernel and re-run the notebook.

In [1]:
from datetime import date
from finmarkets import DiscountCurve

curve = DiscountCurve(
    
    # today date
    date(2016, 1, 1),
    
    # pillar dates
    [
        date(2016, 1, 1),
        date(2016, 6, 1),
        date(2018, 1, 1)    
    ],
    
    # discount factors
    [
        1.0,
        0.98,
        0.82        
    ]
    
)

curve.df(date(2016, 7, 1))

0.970990720891538

## Overnight Index Swaps

Overnight Index Swaps (OIS) are products which pay a floating coupon, determined by overnight rate fixings over the reference periods, against a fixed rate coupon. We will always look at these products from the point of view of the **reciever of the floating leg**.

Therefore an OIS is defined by
* a notional amount $N$
* a start date $d_0$
* a sequence of payment dates $d_1, ..., d_n$
* a fixed rate $K$

For simplicity we're assuming that the fixed and floating legs have the same notional and payment dates - this is not necessarily always the case in practice.

At each payment date, the floating leg pays a cash flow determined as follows:

$$
\def\overnight{\mathrm{o/n}}
\def\ois{\mathrm{ois}}
\def\float{\mathrm{float}}
\def\fixed{\mathrm{fix}}
\def\npv{\mathrm{NPV}}
\def\E{\mathbb{E}}
f_{\float, i} = N \left\{ \prod_{d=d_{i-1}}^{d=d_i-1} \left( 1 + r_\overnight(d) \cdot \frac{1}{360} \right) - 1 \right\}
$$

(This formula is valid for an EONIA swap - i.e. for OIS swaps in EUR - other currencies might have different conventions. The $\frac{1}{360}$ fraction appears because EONIA rates are quoted using the ACT/360 daycount convention and here we're making a simplifying assumption of ignoring weekends and holidays, so we assume that each overnight rate is valid for only one day.)

The sum of the discounted expected values of these cashflows is

$$
\npv_\float = \sum_{i=1}^n P(d_i) \E[f_{\float, i}]
$$

where $P(d)$ is the discount factor with expiry $d$. On the other hand, by definition, we also have the following relationship

$$
\E[f_{\float, i}] = \frac{P_\ois(d_{i-1})}{P_\ois(d_i)} - 1
$$

where $P_\ois(d)$ is the discount factor implied by OIS prices.

In a previous theory lesson we mentioned that the correct curve to use for discounting the flows of a collateralized contract is the one associated with the collateral. Since OIS contracts are collateralized with cash, and cash accrues daily interest at the overnight rate, the OIS curve is itself the correct curve with which to discount the flows of an OIS contract!

In summary, $P = P_\ois$ so the NPV simplifies to

$$
\npv_\float = N \sum_{i=1}^n \left[ P(d_{i-1}) - P(d_i) \right] = N \cdot [ P(d_0) - P(d_n) ]
$$

Each cash flow of the fixed leg is equal to

$$
f_{\fixed, i} = N K \frac{d_i - d_{i-1}}{360}
$$

so the NPV of the fixed leg is

$$
\npv_\fixed = N K \sum_{i=1}^n P(d_i) \frac{d_i - d_{i-1}}{360}
$$

Ultimately the aim will be to take a series of OIS quotations, and determine the discount factors implied by their prices. To do this we'll build a pricing function (or rather, a class), which takes discount curve as the **input** and produces the net present value of the OIS as the **output**. Next lesson we'll put this function inside a numerical optimizer to invert the process.

In [2]:
class OvernightIndexSwap(object):
    
    # this method is called to build the instance, we take some data arguments and save them as attributes of self
    # n.b.: payment_dates should be a list of dates, including the start date as the first element
    def __init__(self, notional, payment_dates, fixed_rate):
        
        self.notional = notional
        self.payment_dates = payment_dates
        self.fixed_rate = fixed_rate
        
    # this method takes a discount curve and calculates the NPV of the floating leg using that curve
    def npv_floating_leg(self, discount_curve):
        
        return self.notional * (
            discount_curve.df(self.payment_dates[0]) -     # self.payment_dates[0] is the start date of the swap
            discount_curve.df(self.payment_dates[-1])      # self.payment_dates[-1] is the last payment date of the swap
        )
    
    # this method takes a discount curve and calculates the NPV of the fixed leg using that curve
    def npv_fixed_leg(self, discount_curve):
        
        npv = 0
        
        for i in range(1, len(self.payment_dates)):   # we loop from i=1 up to but not including the length of the date list
            
            start_date = self.payment_dates[i-1]    # we can do i-1, because the loop starts with i=1
            end_date = self.payment_dates[i]
            
            tau = (end_date - start_date).days / 360
            df = discount_curve.df(end_date)
            
            npv = npv + df * tau
            
        return self.notional * self.fixed_rate * npv
    
    # this method calculates the NPV of the OIS swap
    # n.b.: inside this method we call the other two methods of the class on the same instance 'self',
    #       using self.npv_XXX_leg(...), and we pass the discount_curve we received as an argument
    def npv(self, discount_curve):
        
        float_npv = self.npv_floating_leg(discount_curve)
        fixed_npv = self.npv_fixed_leg(discount_curve)
        
        return float_npv - fixed_npv
            

Now that we have defined the OIS product class, we can build one like so:

In [3]:
from datetime import date

ois = OvernightIndexSwap(
    
    # the notional, one million
    1e6,
    
    # the list of product dates, i.e. the start date then the payment dates
    [
        date(2016, 1, 1),
        date(2016, 4, 1),
        date(2016, 7, 1),
        date(2016, 10, 1),
        date(2017, 1, 1),
    ],
    
    # the fixed rate, 2.5%
    0.025
    
)

Previously in this notebook we built a discount curve an saved it in a variable called `curve`. Let's now use that curve to evaluate the NPV of the OIS.

In [4]:
ois.npv(curve)

58210.785760378596